### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv

grooq_api_key=os.getenv("GROOQ_API_KEY")
os.environ["GROQ_API_KEY"] = "Enter your key"
 

In [14]:
from langchain_groq import ChatGroq ## Langchain integrates with every LLM plateform
model=ChatGroq(model="gemma2-9b-it")
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002353B6AD090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002353B6ADC60>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [17]:
from langchain_core.messages import HumanMessage , SystemMessage

message =[
    SystemMessage(content="Translate the follwoing from English To French"),
    HumanMessage(content="Hellow How are you")
]

result = model.invoke(message)

In [18]:
result

AIMessage(content='The French translation of "Hello, how are you?" is:\n\n**Bonjour, comment allez-vous ?** \n\n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **Comment allez-vous ?** How are you? (formal, polite)\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 23, 'total_tokens': 81, 'completion_time': 0.105454545, 'prompt_time': 0.000175409, 'queue_time': 0.015410261, 'total_time': 0.105629954}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cbbbc585-be8d-495a-b0d4-1fff031cf67b-0', usage_metadata={'input_tokens': 23, 'output_tokens': 58, 'total_tokens': 81})

In [19]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)


'The French translation of "Hello, how are you?" is:\n\n**Bonjour, comment allez-vous ?** \n\n\nHere\'s a breakdown:\n\n* **Bonjour:** Hello\n* **Comment allez-vous ?** How are you? (formal, polite)\n'

In [ ]:
### Using LCEL - chain the component
## Similar to pipeline

chain=model|parser
chain.invoke(message)

"Hello - Bonjour\n\nHow are you? - Comment allez-vous? (formal) or Comment vas-tu? (informal) \n\n\nLet me know if you'd like me to translate any other phrases! \n"

In [21]:
### Prompt Template

from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system" , generic_template),
     ("user" , "{text}")]
)

In [23]:
result=prompt.invoke({"language":"French" , "text":"Hello how are you? I am doing well"})

In [24]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello how are you? I am doing well', additional_kwargs={}, response_metadata={})])

In [25]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello how are you? I am doing well', additional_kwargs={}, response_metadata={})]

In [27]:
chain=prompt|model|parser
chain.invoke({'language' : 'French' , 'text' : 'Hello'})

'Bonjour \n'